In [1]:
from __future__ import print_function, absolute_import
from sklearn.metrics.cluster import normalized_mutual_info_score
import os
import elastix
import matplotlib.pyplot as plt
import numpy as np
import imageio
import SimpleITK as sitk
import voxelmorph as vxm
import fileinput
import pandas as pd
import cv2
import time 
import itk

In [2]:
begin = time.time()
ELASTIX_PATH = os.path.join(r"C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe")
TRANSFORMIX_PATH = os.path.join(r"C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\transformix.exe")
if not os.path.exists(ELASTIX_PATH):
    raise IOError('Elastix cannot be found, please set the correct ELASTIX_PATH.')

el = elastix.ElastixInterface(elastix_path=ELASTIX_PATH)

In [3]:
# paths
# make a folder called 'atlas_images' and copy the 15 folders of the training data in the atlas folder
parameter_file_path = r"C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt"
parameter_file_path_bspline = r"C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt"
data_path = r"C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData" #path to training data folder
atlas_path = r"C:\Users\20192059\Documents\Master\CS in MIA\atlas_images"#path to folder where atlas images are saved
MRI = "mr_bffe.mhd"
atlas  = "prostaat.mhd"

In [4]:
# function for NMI between images
def NMI(img1,img2):
    img1_ = sitk.GetArrayFromImage(img1)
    img2_ = sitk.GetArrayFromImage(img2)
    img2_ = np.reshape(img2_, -1)
    img1_ = np.reshape(img1_, -1)
    normalized_mutual_infor = normalized_mutual_info_score(img1_, img2_)
    nmi = normalized_mutual_infor 
    
    return nmi

In [5]:
# make df for NMI scores and make list with all patients
nmi_df = pd.DataFrame(columns =['Patient','NMI score'])
patient_list = ["p102","p107","p108","p109","p115","p116","p117","p119","p120","p125","p127","p128","p129","p133","p135"]

In [6]:
# for loop over the patients, in which the selected patient is the gold standard
for i in patient_list:
    
    #empty df
    nmi_df.drop(nmi_df.index[:], inplace=True)
    
    #remove existing files in folder
    dir = os.path.join(atlas_path, i) 
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))
    
    #specify validation image
    fixed_image_path = os.path.join(data_path, i, MRI) 
    fixed_image_path_new = os.path.join(data_path, i, atlas) 
    fixed_image_new = imageio.v2.imread(fixed_image_path_new)[40,:,:]
    
    #make list of all patients except validation patient
    training_patients_list = []
    for j in patient_list:
        if i != j:
            training_patients_list.append(j)
    
    #loop over training patients 
    for k in training_patients_list:
        moving_image_path = os.path.join(data_path, k, MRI) #training image
        result_path = os.path.join(atlas_path, i)
        
        #affine registration
        el.register(
            fixed_image=fixed_image_path,
            moving_image=moving_image_path,
            parameters=[parameter_file_path],
            output_dir=result_path)
        
        #alter parameter file after registration
        filename = os.path.join(result_path, 'TransformParameters.0.txt')
        with fileinput.FileInput(filename, backup = '', inplace = True) as f:
            for line in f:
                if("(FinalBSplineInterpolationOrder 3)\n" == line):
                    print("(FinalBSplineInterpolationOrder 0)",end ='\n')
                else:
                    print(line, end ='')
                    
        #change name parameter file            
        old_name = os.path.join(result_path, 'TransformParameters.0.txt')
        new_name = os.path.join(result_path, 'TransformParameters.'+k+'.txt')                    
        os.rename(old_name,new_name)
        
        # transformed moving image
        result_path_affine = os.path.join(result_path, 'result.0.mhd')

        #NMI, select the best images
        img1 = sitk.ReadImage(fixed_image_path,sitk.sitkFloat32)
        img2 = sitk.ReadImage(result_path_affine,sitk.sitkFloat32)
        score = NMI(img1, img2)
        new_row = {'Patient':k, 'NMI score':score}
        nmi_df = nmi_df.append(new_row, ignore_index=True)


    #save nmi_df:
    path_df = os.path.join(atlas_path, i, 'NMI_list.txt')
    nmi_df = nmi_df.sort_values('NMI score')
    with open(path_df, 'a') as f:
        df_string = nmi_df.to_string()
        f.write(df_string)
    

    #Bspine registration of the selected images
    top_patients_list = nmi_df['Patient'].values
    for l in top_patients_list:
        result_path = os.path.join(atlas_path, i)
        moving_image_path = os.path.join(data_path, l, MRI)
        moving_image_path_new = os.path.join(data_path, l, atlas)
        
        tr =elastix.TransformixInterface(parameters=os.path.join(result_path, 'TransformParameters.'+l+'.txt'),
        transformix_path=TRANSFORMIX_PATH)
        
        transformed_image_path = tr.transform_image(moving_image_path, output_dir=result_path)
        transformed_image_path_new = tr.transform_image(moving_image_path_new, output_dir=result_path)
    
        el.register(
            fixed_image=fixed_image_path,
            moving_image=transformed_image_path,
            parameters=[parameter_file_path_bspline],
            output_dir=result_path)
        
        
        #alter parameter file after registration
        filename = os.path.join(result_path, 'TransformParameters.0.txt')
        with fileinput.FileInput(filename, backup = '', inplace = True) as f:
            for line in f:
                if("(FinalBSplineInterpolationOrder 3)\n" == line):
                    print("(FinalBSplineInterpolationOrder 0)",end ='\n')
                else:
                    print(line, end ='')

        #transform atlas image and save
        tr_2 =elastix.TransformixInterface(parameters=os.path.join(result_path, 'TransformParameters.0.txt'),
        transformix_path=TRANSFORMIX_PATH)
        transformed_image_path_2 = tr_2.transform_image(transformed_image_path_new, output_dir=result_path)

        transformed_moving_image_new = itk.imread(transformed_image_path_2.replace('dcm', 'tiff'))#imageio.imread(transformed_image_path_2.replace('dcm', 'tiff'))
        outputFile = os.path.join(result_path, l+'_atlas'+'.mhd')
        itk.imwrite(transformed_moving_image_new, outputFile)
        
        
#         fig, ax = plt.subplots(1, 2, figsize=(20, 5))
#         ax[0].imshow(fixed_image_new, cmap='gray')
#         ax[0].set_title('Fixed image')
#         ax[1].imshow(transformed_moving_image_new, cmap='gray')
#         ax[1].set_title('Moving image')
#         plt.show()
        
end = time.time()
print(f"Total runtime of the program is {end - begin}") 

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p107\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p102
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p108\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p102
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -m C:\Users\2019

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p102\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p102
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p102\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p102
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p107\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p107\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p107
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p107\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p107\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p107
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p107\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p108\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p120\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p108
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p108\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p125\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p108
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p108\mr_bffe.mhd -m C:\Users\2019

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p109\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p109
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p109\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p107\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p109
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p109\mr_bffe.mhd -m C:\Users\2019

...

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p129\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p129
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p129\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p107\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p129
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p129\mr_bffe.mhd -m C:\Users\2019

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p129\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p129\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p129
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p129\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p129\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p129
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p129\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p133\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p133\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p133
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p133\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p133\result.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p133
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p133\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\

C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p135\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p119\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p135
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p135\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p120\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p135
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p135\mr_bffe.mhd -m C:\Users\2019

Total runtime of the program is 9254.793936729431
